# Template for Creating Sacrament Meeting Programs

## Uses MarkupPy to generate the HTML

In [25]:
#
# Sugar Land 2nd Ward HTML Program Writer
#
# Pete Slater
# June 2022

import datetime as dt
tic = dt.datetime.now()
import hymndict # Titles and links to hymns
import htmlpy # Local copy of source code
from htmlpy import oneliner as e
import artlinks # Links to artwork
import sundays # Sunday schedule
import sys

import calendar

# Customized functions to create the HTML for standard agenda items

In [26]:
'''
Define functions for formatting each of the items that can appear in a program.
Build the program in the main script by calling the functions.

'''
def officer(role, officer, page):
    page.p(e.b(role+":")+" "+officer,align="center")

def pagetitle(unit, meeting, meetdate,page):
    page.h1(unit, align="center")
    page.h2(meeting, align="center")
    page.h2(meetdate, align="center")

def speaker(name, page):
    page.h2("Speaker", align="center")
    page.p(name,align="center")
    
def prayer(name, description, page):
    page.h2(description, align="center")
    page.p(name,align="center")    
    
def testimony(name, page):
    page.h2("Testimony", align="center")
    page.p(name,align="center")
    
def testimonies(page):
    page.h2("Bearing of Testimonies", align="center")

def music(number, description, page):
    page.h2(description,align="center")
    [hymntitle, hymnurl] = hymndict.hymns[number]
    page.p(e.a("#" + str(number)+", "+hymntitle, href=hymnurl),align="center")
    
def specialmusic(performers,page,title=None,accompanist=None):
    page.h2("Special Musical Number",align="center")
    if title != None:
        page.p(e.i(title),align="center")
    page.p(performers,align="center")
    if accompanist != None:
        page.p("Acc. by " + accompanist, align="center")
        
def thought(text, author, page):
    page.hr()
    page.p(text, align="center")
    page.p("- "+author,align="center")
 
# Announcements are passed as a list of text strings
def announcements(textlist,page):
    page.hr()
    page.h1("Announcements", align="center")
    for txt in textlist:
        page.p(txt,align="center")
        
# Calendar items are passed as a list of text strings
def print_calendar(textlist,page):
    page.hr()
    page.h1("Calendar Items", align="center")
    for txt in textlist:
        page.p(txt,align="center")

# Custom section for Preparedness Corner, etc.
def custom(title, txtlist, linklist, page):
    page.hr()
    page.h1(title, align="center")
    if txtlist != None:
        for txt in txtlist:
            page.p(txt,align="center")
    if linklist != None:
        for link in linklist:
            page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Place some links, passed as a list containing text and url
def links(linklist, page):
    page.hr()
    page.h1("Links", align="center")
    for link in linklist:
        page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Print credits at the bottom of the page
def credit(editor, page):
#'''Print credits and datestamp ''' 
    today = dt.date.today()
    page.hr()
    text = today.strftime("Edited %B %d, %Y by ") + editor
    page.small(text,align="center")        
    

## Functions for the cleaning calendar

In [27]:
# Enable of checking of whether a date is in our ward's turn
start_turn = dt.date(2023,7,1)
end_turn = dt.date(2024,6,30)

def our_turn(check):
    if check >= start_turn and check <= end_turn:
        return True
    else:
        return False
    
def nextsunday(n):
#""" Return next n Sundays, including today if it's Sunday"""
 today = dt.date.today()
 daynum = today.weekday() # Sunday is day #6

 if daynum == 6:
  basedate = today 
 else:
  basedate = today + dt.timedelta(6-daynum)
    
 sundays = []
 for i in range(n):
  nxtSunday = basedate + dt.timedelta(i*7)
  #sundays.append(nxtSunday.strftime("%B %d, %Y"))
  sundays.append(nxtSunday.strftime("%B %d"))
 return sundays

def nextsaturday(n):
#""" Return next n Saturdays, including today if it's Saturday"""
 today = dt.date.today()
 daynum = today.weekday() # Saturday is day #5

 if daynum == 5:
  basedate = today 
 else:
  basedate = today + dt.timedelta(5-daynum)
    
 saturdays = []
 for i in range(n):
  nxtSaturday = basedate + dt.timedelta(i*7)
  if our_turn(nxtSaturday):
      saturdays.append(nxtSaturday.strftime("%B %d"))
 return saturdays



## Functions for the recurring ward temple days, which as of September 2024, was the 2nd Saturday

In [28]:
# Uses an example from www.w3resource.com

def second_saturday(year, month):
    """get the second Saturday of the month"""
    cal = calendar.monthcalendar(year, month)
    first_week = cal[0]
    second_week = cal[1]
    third_week = cal[2]

    if first_week[calendar.SATURDAY]:
        second_saturday = second_week[calendar.SATURDAY]
    else:
            second_saturday = third_week[calendar.SATURDAY]

    
    #s = "{0} {1}\n".format(calendar.month_name[month], second_saturday)
    
    return dt.date(year, month, second_saturday)

def upcoming_Saturdays(n):
     """return the next n upcoming 2nd Saturdays"""
     
     # Check the date and determine if we're past this month's temple day
     today = dt.datetime.today()
     thismonth = today.month
     thisyear = today.year
     thisdate = today.day
     thismonth_templeday = second_saturday(thisyear, thismonth)
     if (thismonth_templeday.day < thisdate): # This month's temple day is past
          thismonth += 1 # Look at next month
          if thismonth == 13: # Fix potential run into next year
               thismonth = 1
               thisyear += 1
    
    # Get set to return a list of upcoming dates
     result = []

    # Now look at months month ahead, fixing year overflow if necessary
     for i in range(n):
        nextmonth = thismonth+i
        if nextmonth > 12:
            nextmonth -= 12
        if nextmonth < thismonth:
            nextyear = thisyear + 1
        else:
            nextyear = thisyear
        result.append(second_saturday(nextyear, nextmonth))

     output = ["{0} {1}".format(calendar.month_name[trip.month], trip.day) for trip in result]
     output2 = "Upcoming temple days: " + ", ".join(output)

     return(result, output2)

### Cleaning Roster for 2023-24 

In [29]:
roster = {
    "June 24" : "Sugar Land 1st",
    "July 01" : "Bradford, Arnold, Emerson, Markwalter",
    "July 08" : "Clark, Lambert, Blackburn, Ganci, Martin, Robinson",
    "July 15" : "Crandall, Blanco, Garcia, Moline, Thibault",
    "July 22" : "Crosby, Boyer, Griffin, Mike & Dawnette Moore, MacKinnon",
    "July 29" : "Maplewood 3rd Branch",
    "August 05" : "Draney, Buchanan, Hettinger, Morton, Tracey",
    "August 12" : "Leavitt, Chen, Hintze, Ozomah, Seegmiller",
    "August 19" : "Powell, Clements, Horowitz, Quam, Wightman",
    "August 26" : "Kaye Reynolds, Debow, Jeffery, Rasmussen, Winningham",
    "September 02" : "Roskelley, Ekstrom, Ocampo, Juarez, Jason & Deann Moore, Slater",
    "September 09" : "Arnold, Bradford, Emerson, Markwalter",
    "September 16" : "Blackburn, Clark, Ganci, Lambert, Martin, Robinson",
    "September 23" : "Blanco, Crandall, Garcia, Moline, Thibault",
    "September 30" : "Maplewood 3rd Branch",
    "October 07" : "Boyer, Crosby, Griffin, MacKinnon, Mike & Dawnette Moore",
    "October 14" : "Buchanan, Draney, Hettinger, Morton, Tracey",
    "October 21" : "Chen, Hintze, Leavitt, Ozomah, Seegmiller",
    "October 28" : "Clements, Horowitz, Powell, Quam, Wightman",
    "November 04" : "Debow, Jeffery, Rasmussen, Kay Reynolds, Winningham",
    "November 11" : "Ekstrom, Juarez, Jason & Deann Moore, Ocampo, Roskelley, Slater",
    "November 18" : "Arnold, Bradford, Emerson, Markwalter",
    "November 25" : "Blackburn, Clark, Ganci, Lambert, Martin, Robinson",
    "December 02" : "Blanco, Crandall, Garcia, Moline, Thibault",
    "December 09" : "Boyer, Crosby, Griffin, MacKinnon, Mike & Dawnette Moore",
    "December 16" : "Buchanan, Draney, Hettinger, Morton, Tracey",
    "December 23" : "Chen, Hintze, Leavitt, Ozomah, Seegmiller",
    "December 30" : "Maplewood 3rd Branch",
    "January 06" : "Clements, Horowitz, Powell, Quam, Wightman",
    "January 13" : "Debow, Jeffery, Rasmussen, Kay Reynolds, Winningham",
    "January 20" : "Ekstrom, Juarez, Jason & Deann Moore, Ocampo, Roskelley, Slater",
    "January 27" : "Yet to be assigned",
    "February 03" : "Yet to be assigned",
    "February 10" : "Yet to be assigned",
    "February 17" : "Yet to be assigned",
    "January 27" : "Bradford,  Arnold, Gutiérrez,  Markwalter,  Scott",
    "February 03" : "Clark,  Lambert,  Blackburn,  Martin,  Quam,  Robinson",
    "February 10" : "Crandall,  Blanco,  Ganci,  Garcia,  Moline,  Thibault",
    "February 17" : "Crosby,  Boyer,  Griffin, Mike & Dawnette Moore,  Thomas",
    "February 24" : "Draney,  Buchanan,  Hettinger,  Morton,  Tracey",
    "March 02" : "Leavitt,  Chen,  Hintze,  Ozomah,  Seegmiller",
    "March 09" : "Powell,  Clements,  Horowitz,  Rellaford,  Wightman",
    "March 16" : "Kaye  Reynolds,  Debow,  Jeffery,  Rasmussen,  Winningham",
    "March 23" : "Roskelley,  Ekstrom,  Ocampo,  Evans, Kyle & Vanessaa Moore,  Slater",
    "March 30" : "Maplewood 3rd  Branch",
    "April 06" : "Bradford,  Arnold,  Guiterrez,  Markwalter,  Scott",
    "April 13" : "Clark,  Lambert,  Blackburn,  Martin,  Quam,  Robinson",
    "April 20" : "Crandall,  Blanco,  Ganci,  Garcia,  Moline,  Thibault",
    "April 27" : "Crosby,  Boyer,  Griffin, Mike & Dawnette Moore,  Thomas",
    "May 04" : "Draney,  Buchanan,  Hettinger,  Morton,  Tracey",
    "May 11" : "Leavitt,  Chen,  Hintze,  Ozomah,  Seegmiller",
    "May 18" : "Powell,  Clements,  Horowitz,  Rellaford,  Wightman",
    "May 25" : "Kaye  Reynolds,  Debow,  Jeffery,  Rasmussen,  Winningham",
    "June 01" : "Roskelley,  Ekstrom,  Ocampo,  Evans, Kyle & Vanessaa Moore,  Slater",
    "June 08" : "Bradford, Arnold, Markwalter, Scott",
    "June 15" : "Clark, Blackburn, Martin, Quam, Robinson",
    "June 22" : "Crandall, Blanco, Ganci, Garcia, Moline, Thibault",
    "June 29" : "Maplewood 3rd Branch",
    "July 07" : "Sugar Land 1st Ward",
    "July 14" : "Sugar Land 1st Ward",
    "July 21" : "Sugar Land 1st Ward",
    "ZZZ" : "End of List"
}

# Determine the Sunday date and what type of meeting

In [30]:
keydate = nextsunday(1)[0]
[meeting, format_date] = sundays.sunday_type(keydate)

# Format the agenda items that are always present
Customize for each week with date and type of meeting, i.e. sacrament, fast and testimony, ward conference

## Set this week's hymns here

In [ ]:
hymn_open = 93
hymn_sacrament = 173
hymn_rest = 96
hymn_close = 94

In [32]:
title_choices = {'Sacrament Meeting': "Sugar Land Second Ward", 'Stake Conference': "Houston Texas South Stake", 
                 "Primary Program" : "Sugar Land Second Ward", 'Ward Conference' : 'Sugar Land Second Ward'}
Title = title_choices.get(meeting, 'Sugar Land Second Ward')

#header_choices = {'Normal': "Sacrament Meeting", 'Fast': "Fast and Testimony Meeting",
#                  'Stake': "Stake Conference", "General" : "General Conference", "Ward" : "Ward Conference"}
#header = header_choices.get(meeting, 'Sacrament Meeting')

footer = ""
styles = ( 'layout.css', 'alt.css', 'images.css' )

page = htmlpy.page( )
page.init()
page.br( )
 
# Make sure it will look good on all devices
page.meta(name="viewport", content="width=device-width, initial-scale=1.0")

# Define the elements on the current week's programs here

pagetitle(Title, meeting, format_date, page)

## Artwork
Pick an artwork from the defined list of links in artlinks.py and enter the number here

Christmas-themed keys are "Nativity1", "Annunc", "Birth", "Simon"

In [33]:
# Place an artwork from the imported dictionary of links
#page.p(e.img(width=299, height=300*0.8, src=artlinks.art[2]), align="center")
page.p(e.img(style="max-width:50%;height:auto;", src=artlinks.art[6]), align="center")

## Officers

In [34]:
if meeting in ['Sacrament Meeting','Fast and Testimony Meeting', 'Primary Program', 'Ward Conference']:
    #officer("Presiding","President Travis Bird", page)
    #officer("Presiding","President Mike Kennington", page)
    #officer("Presiding","President Ernie Carrasquillo", page)
    officer("Presiding","Bishop Cody Roskelley", page)
    #officer("Conducting","Bishop Cody Roskelley", page)
    #officer("Presiding","Brother Jared Draney", page)
    officer("Conducting","Brother Jared Draney", page)
    #officer("Presiding","Brother Schuyler Robinson", page)
    #officer("Conducting","Brother Schuyler Robinson", page)


## Opening and sacrament hymns

In [35]:
if meeting in ["Sacrament Meeting", "Fast and Testimony Meeting", "Ward Conference", "Easter Service", 'Primary Program']:
    music(hymn_open,"Opening Hymn",page)
    #prayer("Jack Reynolds", "Invocation", page)
    music(hymn_sacrament,"Sacrament Hymn", page) 

## Variable part of the program

### Fast and Testimony meeting

In [36]:
# Uncomment this line for fast and testimony meeting - logic should now handle this
if meeting == "Fast and Testimony Meeting":
    testimonies(page)

### Sacrament meeting or ward conference

In [37]:
if meeting in ["Sacrament Meeting", "Ward Conference", "Easter Service"]:
    # Use lines like these to form the speaking part of the program
     
    #specialmusic("Primary and Young Women", page, title = "More Than Enough")
    speaker("Claire Draney",page)
    speaker("Wendy Markwalter", page)
    #speaker("Kaycee Crandall", page)
    
    music(hymn_rest,"Rest Hymn", page)

    speaker("Mike Moore",page)
    


### Stake Conference

In [38]:
if meeting == "Stake Conference":
    # Special Code for Stake Conference
    txtlist = (
        "All members of Stake and Ward Councils and their presidencies, secretaries, and clerks are invited to attend.",
        "This includes: Bishoprics; Elders quorum presidencies; Aaronic Priesthood quorum and Young Women class advisers; Relief Society, Young Women, Primary and Sunday School presidencies, and secretaries.",
        "In-person at Lexington"
    )
    linklist = None

    custom("Leadership Session, Saturday 4:00-5:45 pm", txtlist, linklist, page)

    txtlist = (
         "All Adults",
         "In-person at Lexington"
    )
    linklist = None

    custom("Adult Session, Saturday 7:00-8:45 pm", txtlist, linklist, page)

    txtlist = (
        "In-person at Lexington",
        "Via Zoom at Sienna and Rosenberg"
    )
    linklist = None

    custom("General Session, Sunday 10:00 am - 12:00 pm", txtlist, linklist, page)

### General Conference

In [39]:
if meeting == "General Conference":
    # Special Code for General Conference
    txtlist = (
        "Everyone of all faiths, beliefs, and background is invited to watch, listen, and participate in General Conference.",
        "",
    )
    
    
    page.h1("General Conference", align="center")
    page.h2(txtlist, align = "center")
       

    txtlist = (
         "Saturday Morning Session - 11:00 am CDT",
         "Saturday Afternoon Session - 3:00 pm CDT",
         "Saturday Evening Session - 7:00 pm CDT",
         "Sunday Morning Session - 11:00 am CDT",
         "Sunday Afternoon Session - 3:00 pm CDT",
    )
    linklist = {
        ("Choose a stream to watch or listen","https://www.churchofjesuschrist.org/feature/general-conference?lang=eng")
    }
    custom("Join General Conference", txtlist, linklist, page)

    

### Primary Program

In [40]:
if meeting == "Primary Program": 
# Special code for the Primary and Christmas Programs
    txtlist = (
        'Words and Music Testifying of the Book of Mormon, Another Testament of Jesus Christ',
        'Directed by Dawnette Moore, Accompanied by Dana Blackburn',
        'Primary Choir with Porter Wrightman, Maggie Moore, Ryan Boyer, Livia Moore, Evangelina Garcia, Theodore Rellaford, Boston Robinson, Piper Wrightman, Bella Moore, Sebastian Garcia, Gresham Robinson, Eliza Moore, Leonor Chen, Ioanna Ozomah, Dylan Moore, Henrie Rellaford',
        'Concluding remarks by Bishop Joey Powell'
        )
    linklist = None

    custom("Primary Program", txtlist,linklist, page)

### Closing hymn and prayer

In [41]:
if meeting in ["Sacrament Meeting", "Fast and Testimony Meeting", "Ward Conference", "Easter Service", "Primary Program"]:
    music(hymn_close,"Closing Hymn",page)
    #prayer("","Benediction",page)

## End matter - thought, announcements, calendar, links

In [42]:
# Spiritual thought
#quote = ("The Lord isn't asking us to load up a handcart; He's asking us to fortify our faith.")
#author = 'Elder M. Russell Ballard'
#quote = "General conference provides an opportunity to receive personal revelation as general Church leaders give counsel and direction from the Lord. We encourage all to listen to, study, ponder, and apply the counsel and direction given"
#author = "The First Presidency, September 5, 2024"
#quote = "Tithing is a principle that is fundamental to the personal happiness and well-being of the Church members worldwide, both rich and poor. Tithing is a principle of sacrifice and a key to the opening of the windows of heaven."
#author = "James E. Faust"
#quote = "When so many around us are burdened with fear and uncertainty, I invite you to make room in your heart for those around you who may be struggling to see the light of the Savior and to feel His love. No gifts will mean as much as acts of pure love you offer to the lonely, the worn down and the weary. These are gifts that remind us and them of the true reason for the season: the gift of God’s Son, Jesus Christ, who was born to cast out all fear and bring everlasting light and joy to all who follow Him"
#author = "Russell M. Nelson"
import quotelib

thought(quotelib.text, quotelib.author, page)

# Make a list of announcements, then post to the page
txtlist = (
        
        "Volleyball Wednesdays - Come join us on Wednesdays at 9PM and have fun!!!",  
        "Send announcements to bvl2clerk@gmail.com")
announcements(txtlist, page)

# Post a Flyer
# Houston South Stake puts out a monthly flyer with calendar items. We post the updates to the same file when they arrive
page.p(e.img(style="max-width:50%;height:auto;", src='https://sugar-land-2nd-ward-programs.github.io/Stake%20Monthly%20Flyer.jpg'), align="center")
page.p(e.img(style="max-width:50%;height:auto;", src='https://sugar-land-2nd-Ward-Programs.github.io/Stake%20Save%20the%20Date.jpg'), align="center")
page.p(e.img(style="max-width:50%;height:auto;", src='https://sugar-land-2nd-ward-programs.github.io/StartingGrowingSmallBusinessOrientation%20Flyer.jpg'), align="center")
# The Chosen Fireside - Remove after 23 March 2025
#page.p(e.img(style="max-width:50%;height:auto;", src='https://sugar-land-2nd-ward-programs.github.io/The%20Chosen%20Flyer.jpg'), align="center")


#page.p(e.a("Sign up for the Christmas Party on December 7", href="https://www.signupgenius.com/go/10C0448A9A82EAB9-53466299-christmas"),align="center")

# Ward temple days now get their own line item
upcoming, string_result = upcoming_Saturdays(6)

callist = (
    string_result,
    "Stake Conference, May 31 - June 1.",
    "Seminary Graduation, June 1, 6:30 pm",
    "Beehive Camp. June 5-7, Sugar Land",
    "June 11, Astros Friends & Family Night",
    "Multistake Fort Bend County Disaster Devotional, June 11, 7:00 pm, Richmond Stake Center",
    "The Lexington Building will be closed for cleaning on July 15.",
    "The Lexington Cultural Hall will be closed for maintenance July 28 - August 3"       
            )
print_calendar(callist, page)


# Cleaning Assignments

In [43]:
if (our_turn(dt.date.today())):
    sats = nextsaturday(4)
    cleaninglist = []
    for sat in sats:
        cleaninglist.append(sat + ": " + roster[sat])
    custom("Cleaning Assignments",cleaninglist, None, page)

### Preparedness Corner - under development

In [44]:
txtlist = (
"Hurrican Season begins June 1. It's time to get your household ready.",
    )
#linklist = {
#    ("Attend the Ready Fort Bend Expo on May 3","https://www.facebook.com/events/s/ready-fort-bend-expo-2025/581080474944743/?rdid=Zcfv8aoLnKy8tEw5&share_url=https%3A%2F%2Fwww.facebook.com%2Fshare%2F1BV8pnNt1w%2F#"),
#}
linklist = None
custom("Emergency Preparedness Corner", txtlist,linklist, page)

In [ ]:
# Make a list of links, then post
linklist = (
        ("Register for the Zoom Personal Finance Class","https://quickreg.churchofjesuschrist.org/join-now/group/e8324fd4-ef16-f011-998a-7c1e52821a58"),
        ("Share Activities with the Ward Web Page","https://local.churchofjesuschrist.org/en/units/us/tx/sugar-land-2nd-ward?icid=actsh|cmh|web|mis|global|global"),
        ("Let the missionaries know when you are available to help teach","https://docs.google.com/forms/d/1cVhEINFFESai0MLbgBA8euTyiMQsM8R1Sur_EQsyUrw/edit?usp=drive_web"),
        ("Get started with Temple and Family History work","https://sugar-land-2nd-ward-programs.github.io/TempleandFamilyHistoryGettingStarted.html"),
        ("See to Succeed a Houston South Stake service project","https://www.houstonhealth.org/services/clinical/see-to-succeed"),
        ("Sign up to feed the missionaries","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-dinner2"),
        ("Sign up to prepare a meal with Fort Bend Family Promise","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-helping4#"),
        ("Help at secondmile.org","https://secondmile.org"), 
        )
links(linklist, page)
credit("Pete Slater and Andy Chen", page)

# Write out the file to Index.html, which must be posted online

In [46]:
print (page)# -*- coding: utf-8 -*

original_stdout = sys.stdout
with open('Index.html', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(page)
    sys.stdout = original_stdout # Reset the standard output to its original value

toc = dt.datetime.now()
# find difference loop start and end time and display
td = (toc - tic).total_seconds() * 10**3
print(f"Script executed in  : {td:.03f}ms")
"""
End of script
"""

<!DOCTYPE HTML PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
<html lang="en">
<head>
</head>
<body>
<br />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<h1 align="center">Houston Texas South Stake</h1>
<h2 align="center">Stake Conference</h2>
<h2 align="center">June 1st</h2>
<p align="center"><img style="max-width:50%;height:auto;" src="https://assets.ldscdn.org/02/22/0222d8e4e6f2585f1cb63c25997e367993c25f79/jesus_blessing_the_children.jpeg" /></p>
<hr />
<h1 align="center">Leadership Session, Saturday 4:00-5:45 pm</h1>
<p align="center">All members of Stake and Ward Councils and their presidencies, secretaries, and clerks are invited to attend.</p>
<p align="center">This includes: Bishoprics; Elders quorum presidencies; Aaronic Priesthood quorum and Young Women class advisers; Relief Society, Young Women, Primary and Sunday School presidencies, and secretaries.</p>
<p align="center">In-person at Lexington</p>
<hr />
<h1 align="center">Adult Session, Satu

'\nEnd of script\n'

In [47]:
meeting

'Stake Conference'

In [48]:
keydate

'June 01'